In [1]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('~/Health/ozone_19_mort_est_total(1119).csv')

In [3]:
ozone = data[['loc','longitude','latitude',
              'arcgis_ID','location_id','cont_new_ID',
              'population_total_19','population_urban_19','population_rural_19',
              'MDA8h_urban_warm_19','MDA8h_rural_warm_19',
              'cvd_25_death_urban_est','cvd_25_death_urban_lb','cvd_25_death_urban_ub','cvd_25_death_rural_est','cvd_25_death_rural_lb','cvd_25_death_rural_ub']]
ozone = ozone.rename(columns={'cont_new_ID':'cont_ID'})

In [4]:
age_19 = pd.read_csv('~/ozone-mortality/age structure list/age_19_list.csv')
mort_19 = pd.read_csv('~/ozone-mortality/mortality list/mort_19_list.csv')

In [5]:
ozone_mort = ozone.merge(mort_19, how='left', left_on=['location_id'], right_on=['location_id'])

In [6]:
ozone_mort_age = ozone_mort.merge(age_19, how='left', left_on=['location_id','year','location_name'], right_on=['location_id','year','location_name'])

#### Here we prescribe all necessary parameters: multi-cause exposure-response RRs with 95% CIs, and the low-concentration cut-off (LCC, or threshold). The pooled RRs are estimated by meta-analysis (see a "tutorial" code file). The LCC value(s) are defined as the lowest 5 𝑡ℎ  percentile of the multi-data fused ambient ozone dataset (only considering the residential land areas) following Malley et al. 2017.

In [7]:
lnRR_cvd = np.log(1.017)
lnRR_sigma_cvd = (np.log(1.025) - np.log(1.017))/1.96 
LCC = 40

#### For the sake of quick computation (e.g. less than 10 minutes on JASMIN supercomputation system) as a demonstration, we have simplified some trivial aspects: (1) we cluster the age ≥25 population rather than 5-yr stratification. (2) we do not calculate the arithmetic mean and standard deviations, as we choose to report the median with 95% uncertainty intervals. 

In [8]:
data = ozone_mort_age
start = time.process_time()

for t in range(0,742299,1):
    
    ## Extract the age ≥25 risked proportion 
    pop_all_age       = data.iloc[t,58]
    pop_all_age_sigma = (data.iloc[t,58]-data.iloc[t,59])/1.96
    pop_25_age        = data.iloc[t,97]
    pop_25_age_sigma  = (data.iloc[t,97]-data.iloc[t,98])/1.96
    
    ## Extract the baseline mortality rate 
    cvd_25_mort            = data.iloc[t,19]
    cvd_25_mort_sigma      = (data.iloc[t,19]-data.iloc[t,20])/1.96
    
    ## Extract urban and rural population and MDA8h-ozone exposure 
    pop_urban_grid = data.iloc[t,7] 
    o3_mda8h_urban = data.iloc[t,9] 
    pop_rural_grid = data.iloc[t,8] 
    o3_mda8h_rural = data.iloc[t,10] 

    ## Convert into the hazadous dose from exposure concentration
    o3_mda8h_urban_dose = max(0,o3_mda8h_urban - LCC) 
    o3_mda8h_rural_dose = max(0,o3_mda8h_rural - LCC) 
    
    ## Generate Gaussian distributed samples 
    pop_all_age_dist      = np.random.normal(pop_all_age, pop_all_age_sigma, 1000) 
    pop_25_age_dist       = np.random.normal(pop_25_age,  pop_25_age_sigma,  1000)
    
    cvd_25_mort_dist      = np.random.normal(cvd_25_mort, cvd_25_mort_sigma, 1000)
    
    RR_dist_cvd           = np.random.normal(lnRR_cvd,    lnRR_sigma_cvd,    1000)
    
    ## Start the Monte Carlo bootstrap simulation realisations 
    delta_mort_cvd_urban  = cvd_25_mort_dist * (1 - np.exp(- RR_dist_cvd * o3_mda8h_urban_dose/10)) * pop_urban_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    delta_mort_cvd_rural  = cvd_25_mort_dist * (1 - np.exp(- RR_dist_cvd * o3_mda8h_rural_dose/10)) * pop_rural_grid * pop_25_age_dist / pop_all_age_dist / 100000 
    
    ## Summarise the results    
    est_cvd_urban = np.nanpercentile(delta_mort_cvd_urban,50) 
    lb_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,2.5)
    ub_cvd_urban  = np.nanpercentile(delta_mort_cvd_urban,97.5)

    est_cvd_rural = np.nanpercentile(delta_mort_cvd_rural,50) 
    lb_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,2.5)
    ub_cvd_rural  = np.nanpercentile(delta_mort_cvd_rural,97.5)
    
    ## Store the distribution statistics into the DataFrame 
    data.iloc[t,11]  = est_cvd_urban
    data.iloc[t,12]  =  lb_cvd_urban
    data.iloc[t,13]  =  ub_cvd_urban

    data.iloc[t,14]  = est_cvd_rural
    data.iloc[t,15]  =  lb_cvd_rural
    data.iloc[t,16]  =  ub_cvd_rural
    
    if t%1000 == 0:
        print(t) ## Add a tracer to check the simulation progresses 
    
elapsed = (time.process_time() - start)
print('Time used:', elapsed, "s")    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [9]:
data['cvd_25_death_urban_est'].sum()

365840.43473834463

In [10]:
data['cvd_25_death_urban_lb'].sum()

222723.03162200295

In [11]:
data['cvd_25_death_urban_ub'].sum()

537860.9447667374

In [12]:
data['cvd_25_death_rural_est'].sum()

402990.678291845

In [13]:
data['cvd_25_death_rural_lb'].sum()

245197.20458610376

In [14]:
data['cvd_25_death_rural_ub'].sum()

593173.2005847957

In [15]:
data.to_csv('ozone-cvd-mortality-2019(v0.1).csv', index=False)